# Method 1: (prefered approach) 

**This method uses the Descendents API which I think only works if you are a HUBMAP member, if you are not prefer using the method method 2 using entity api**

In [1]:
import pandas as pd
import requests
import json
import os
from tqdm import tqdm

In [4]:
def load_data(path):
    df = pd.read_csv(path)
    return df[['HuBMAP ID']].dropna().reset_index(drop=True)

input_df = load_data("/u/sbdubey/CLI_HUBMAP/hra-deepcell-experiments/scripts/lymph-codex-ufl/data/deepcell_lymph.csv")
input_df.head()


,HuBMAP ID
0,HBM723.BZKF.992
1,HBM927.KDXB.445
2,HBM283.GNGR.785
3,HBM483.KDBW.482
4,HBM457.JZFF.434


In [10]:
TOKEN = "AgJGeqNv4pYmW9K21al9K2nay0x0ol7dp1YzONgzJzlByDKQXqC0CWmBNMp6yk78DM4k17Jpp115PWtqvdrG1FKBezr"
BASE = "https://entity.api.hubmapconsortium.org/descendants/"

headers = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}

def fetch_descendants(hubmap_id):
    url = BASE + hubmap_id
    try:
        r = requests.get(url, headers=headers, timeout=10)
        if r.status_code == 200:
            return r.json()
        else:
            return None
    except:
        return None


**Note the `if` condition is given because this specific hubmap id has more than on descendents `HBM834.ZFVJ.978` but only one of them is processed**

In [11]:
# added if condition read the markdown for why.
def extract_codex_ids(descendants_json, parent_hubmap_id=None):
    if parent_hubmap_id == "HBM834.ZFVJ.978":
        return ["HBM522.BSZT.385"]

    if not isinstance(descendants_json, list):
        return []

    result = []
    for entry in descendants_json:
        if (
            entry.get("dataset_type") == "CODEX [Cytokit + SPRM]"
            and entry.get("status") == "Published"
            and entry.get("last_modified_user_displayname") == "Karl Burke"  # spleen dataset required this condition
        ):
            result.append(entry.get("hubmap_id"))
    return result

# ...existing code...
for hubmap_id in tqdm(input_df['HuBMAP ID'], desc="Processing"):
    data = fetch_descendants(hubmap_id)
    codex_ids = extract_codex_ids(data, parent_hubmap_id=hubmap_id) if data else []
# ...existing code...

Processing: 100%|██████████| 23/23 [00:14<00:00,  1.63it/s]


In [12]:
output_rows = []

for hubmap_id in tqdm(input_df['HuBMAP ID'], desc="Processing"):
    data = fetch_descendants(hubmap_id)
    codex_ids = extract_codex_ids(data, parent_hubmap_id=hubmap_id) if data else []

    output_rows.append({
        "Input_HuBMAP_ID": hubmap_id,
        "Found_CODEX_IDs": ", ".join(codex_ids) if codex_ids else None
    })

output_df = pd.DataFrame(output_rows)
output_df.head()


Processing: 100%|██████████| 23/23 [00:10<00:00,  2.10it/s]


,Input_HuBMAP_ID,Found_CODEX_IDs
0,HBM723.BZKF.992,HBM332.QJCH.296
1,HBM927.KDXB.445,HBM435.FHMP.758
2,HBM283.GNGR.785,HBM387.QZQF.984
3,HBM483.KDBW.482,HBM673.TNZM.339
4,HBM457.JZFF.434,HBM443.FDHZ.888


In [13]:
output_df

,Input_HuBMAP_ID,Found_CODEX_IDs
0,HBM723.BZKF.992,HBM332.QJCH.296
1,HBM927.KDXB.445,HBM435.FHMP.758
2,HBM283.GNGR.785,HBM387.QZQF.984
3,HBM483.KDBW.482,HBM673.TNZM.339
4,HBM457.JZFF.434,HBM443.FDHZ.888
5,HBM754.WKLP.262,HBM938.TNNT.879
6,HBM834.ZFVJ.978,HBM522.BSZT.385
7,HBM746.ZTBR.275,HBM622.JXWQ.554
8,HBM866.TGNJ.847,HBM268.NKXB.243
9,HBM958.GGVQ.546,HBM992.RHJW.288


In [14]:
output_df.to_csv("descendant_hubmapID_lymph_01.csv", index=False)
print("Saved: codex_results.csv")

Saved: codex_results.csv


**Check if the file is present in Ross folder (I did not run for lymph)**

In [18]:
# Check if Input_HuBMAP_ID values are in processed_ross folder
import glob
import os

# Get list of processed files
processed_files = glob.glob("dataset_hiskers/processed_ross/*.csv")
processed_ids = [os.path.basename(f).split('_')[0] for f in processed_files]

results = []
for _, row in output_df.iterrows():
    input_id = row['Input_HuBMAP_ID']
    
    # Convert ID format: HBM496.ZJFC.554 -> HBM496ZJFC554
    id_no_dots = input_id.replace('.', '')
    
    # Check if matching file exists in processed_ross
    in_processed = id_no_dots in processed_ids
    
    results.append({
        'Input_HuBMAP_ID': input_id,
        'In_Processed_Ross': in_processed,
        'Processed_Filename': f"{id_no_dots}_deepcell_population.csv" if in_processed else None
    })

check_input_df = pd.DataFrame(results)
print(f"Total Input IDs: {len(check_input_df)}")
print(f"Found in processed_ross: {check_input_df['In_Processed_Ross'].sum()}")
check_input_df

Total Input IDs: 64
Found in processed_ross: 0


,Input_HuBMAP_ID,In_Processed_Ross,Processed_Filename
0,HBM235.VKNJ.237,False,None
1,HBM238.GTNW.259,False,None
2,HBM242.LSCK.393,False,None
3,HBM284.SBPR.357,False,None
4,HBM285.VFDT.966,False,None
...,...,...,...
59,HBM945.FSHR.864,False,None
60,HBM953.KMTG.758,False,None
61,HBM964.FPNH.767,False,None
62,HBM974.CNWK.327,False,None


# Method 2: using Entity API (open to all) 
**Note: I did not try this method on this data**

In [1]:
import pandas as pd
import requests
import re
import os
import warnings
from tqdm import tqdm

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)


In [2]:
def load_data(path):
    return pd.read_csv(path)[['HuBMAP ID']].dropna().reset_index(drop=True)

def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")

def get_last_uuid(hubmap_id):
    base_url = "https://entity.api.hubmapconsortium.org/entities/"
    url = f"{base_url}{hubmap_id}"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            uuids = re.findall(r'"uuid"\s*:\s*"([a-f0-9\-]+)"', r.text)
            return uuids[-1] if uuids else None
        return None
    except Exception:
        return None


In [4]:
csv_path = "/u/sbdubey/deepcell_intestine.csv"
df = load_data(csv_path)
df.head()


,HuBMAP ID
0,HBM235.VKNJ.237
1,HBM238.GTNW.259
2,HBM242.LSCK.393
3,HBM284.SBPR.357
4,HBM285.VFDT.966


In [5]:
df['last_uuid'] = [
    get_last_uuid(hid) for hid in tqdm(df['HuBMAP ID'], desc="Fetching UUIDs")
]


Fetching UUIDs: 100%|██████████| 64/64 [01:13<00:00,  1.15s/it]


In [6]:
out_path = "/u/sbdubey/hubmap_uuid_map.csv"
df.to_csv(out_path, index=False)
print(f"Saved: {out_path}")
df.head()


Saved: /u/sbdubey/hubmap_uuid_map.csv


,HuBMAP ID,last_uuid
0,HBM235.VKNJ.237,c8ee4bd8f052b50fab2e9e9365793374
1,HBM238.GTNW.259,a7ecf4fd858a58dcaa3bab5df455065b
2,HBM242.LSCK.393,a702c94500a2f737b722e34f6df7e7dd
3,HBM284.SBPR.357,a9d4471b41cd0d8185e12dd6c5a48e96
4,HBM285.VFDT.966,7034950e109586361e73c1b9ddb81346


# Extract the Descendents

In [7]:
df = pd.read_csv('/u/sbdubey/hubmap_uuid_map.csv')

In [8]:
def fetch_json(uuid):
    url = f"https://portal.hubmapconsortium.org/browse/dataset/{uuid}.json"
    try:
        r = requests.get(url, timeout=10)
        return r.json() if r.status_code == 200 else None
    except:
        return None


In [9]:
def extract_fields(js):
    if js is None:
        return {"total_descendants": None, "first_descendant_id": None}
    
    counts = js.get("descendant_counts", {}).get("entity_type", {})
    total = sum(counts.values()) if isinstance(counts, dict) else None
    
    desc_ids = js.get("descendant_ids", [])
    first_id = desc_ids[0] if len(desc_ids) > 0 else None
    
    return {
        "total_descendants": total,
        "first_descendant_id": first_id
    }


In [10]:
results = []

for uid in tqdm(df["last_uuid"], desc="Fetching Descendants"):
    js = fetch_json(uid)
    fields = extract_fields(js)
    results.append(fields)

desc_df = pd.DataFrame(results)
desc_df.head()


Fetching Descendants: 100%|██████████| 64/64 [00:11<00:00,  5.68it/s]


,total_descendants,first_descendant_id
0,6,10c0c11280c00f324259fe38e2291ee4
1,6,8b21db2002a5179b03532d183a4885eb
2,6,6bdd149dc47782aefdd0e23599708183
3,6,550f3ef14b113c24fd21b8b0750bf078
4,6,aa54aad994ca8a64fa52b3f3945c01b7


In [11]:
final_df = pd.concat([df, desc_df], axis=1)

out_path = "/u/sbdubey/hubmap_descendants_map.csv"
final_df.to_csv(out_path, index=False)

print(f"Saved: {out_path}")
final_df.head()


Saved: /u/sbdubey/hubmap_descendants_map.csv


,HuBMAP ID,last_uuid,total_descendants,first_descendant_id
0,HBM235.VKNJ.237,c8ee4bd8f052b50fab2e9e9365793374,6,10c0c11280c00f324259fe38e2291ee4
1,HBM238.GTNW.259,a7ecf4fd858a58dcaa3bab5df455065b,6,8b21db2002a5179b03532d183a4885eb
2,HBM242.LSCK.393,a702c94500a2f737b722e34f6df7e7dd,6,6bdd149dc47782aefdd0e23599708183
3,HBM284.SBPR.357,a9d4471b41cd0d8185e12dd6c5a48e96,6,550f3ef14b113c24fd21b8b0750bf078
4,HBM285.VFDT.966,7034950e109586361e73c1b9ddb81346,6,aa54aad994ca8a64fa52b3f3945c01b7


**`first_descendant_id` shows the real descendant UUID.
The count is 6 due to some weird/gibberish data in the JSON that I didn’t understand,
but `first_descendant_id` is the correct descendant.**
